In [30]:
import numpy as np
import pandas as pd

1. CCTV 데이터

In [31]:
cctv = pd.read_csv("서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv",
    skiprows=1, encoding = 'EUC-KR')

cctv.head(3)


,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"77,032","7,667","2,200","3,491","4,439","6,582","8,129","9,947","9,876","11,961","11,132","1,608"
1,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
2,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289


In [32]:
cctv.isna().sum().sum()

0

In [33]:
for column in cctv.columns[1:]:
    cctv[column] = cctv[column].apply(lambda x: int(x.replace(',','')))

cctv.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289
3,용산구,2383,34,71,234,125,221,298,351,125,307,617,0
4,성동구,3602,448,125,212,105,339,310,874,390,262,461,76


In [34]:
# 최근 증가율
cctv['최근3개년'] = cctv['2018년'] + cctv['2019년'] + cctv['2020년']
cctv['2017년이전'] = cctv.총계 - cctv['최근3개년'] - cctv['2021년']
cctv['최근증가율'] = (cctv['최근3개년'] / cctv['2017년이전'] * 100).round(2)
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,최근3개년,2017년이전,최근증가율
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608,32969,42455,77.66
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43,294,1435,20.49
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289,902,1142,78.98


In [35]:
cctv['최근증가율2'] = (cctv.iloc[:,9:12].sum(axis=1) / cctv.iloc[:, 2:9].sum(axis=1) * 100).round(2)
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,최근3개년,2017년이전,최근증가율,최근증가율2
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608,32969,42455,77.66,77.66
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43,294,1435,20.49,20.49
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289,902,1142,78.98,78.98


In [36]:
# Selection = 필요한 컬럼 추출
cctv = cctv[['구분', '총계','최근증가율']]
cctv.head(3)


,구분,총계,최근증가율
0,계,77032,77.66
1,종로구,1772,20.49
2,중 구,2333,78.98


In [37]:
# 컬럼명 변경
cctv.rename(columns={'구분' : '구별', '총계' : 'CCTV댓수'}, inplace=True)

In [38]:
# 첫번째 행(계) 삭제
cctv.drop([0], inplace=True)
cctv.head(3)

,구별,CCTV댓수,최근증가율
1,종로구,1772,20.49
2,중 구,2333,78.98
3,용산구,2383,78.64


In [39]:
# 구 이름에서 공백 지우기
# cctv['구별'][2] = '중구'
cctv.구별 = cctv.구별.str.replace(',','')
cctv.head(3)

,구별,CCTV댓수,최근증가율
1,종로구,1772,20.49
2,중 구,2333,78.98
3,용산구,2383,78.64


In [40]:
# CCTV 댓수가 많은 자치구 Top 5
cctv.sort_values('CCTV댓수', ascending=False).head()

,구별,CCTV댓수,최근증가율
23,강남구,6502,61.64
21,관악구,4942,49.91
17,구로구,4075,48.83
8,성북구,3958,94.53
12,은평구,3791,117.86


In [41]:
### 2. 서울시 인구 데이터

In [42]:
pop = pd.read_csv("서울시주민등록인구(구별)통계.txt", sep = '\t', skiprows=2)

In [43]:
pop.isna().sum().sum()

0

In [44]:
# 필요한 컬럼 추출 및 이름 변경
pop = pop[['자치구','계','계.2','65세이상고령자']]
pop.columns = ['구별','인구수','외국인','고령자']
pop.head(3)

,구별,인구수,외국인,고령자
0,합계,"9,795,426","229,436","1,576,971"
1,종로구,"155,106","9,077","27,605"
2,중구,"132,259","9,243","23,980"


In [45]:
# 천단위 구분기호 없애기
for column in pop.columns[1:]:
    pop[column] = pop[column].str.replace(',','').astype(int)

pop.head(3)

,구별,인구수,외국인,고령자
0,합계,9795426,229436,1576971
1,종로구,155106,9077,27605
2,중구,132259,9243,23980


In [46]:
pop.drop([0], inplace=True)

In [47]:
# 외국인 비율, 고령자비율 컬럼만들기
pop['외국인비율'] = (pop.외국인 / pop.인구수 * 100).round(2)
pop['고령자비율'] = (pop.고령자 / pop.인구수 * 100).round(2) 

### 3. 두 개의 데이터 프레임 합치기

In [48]:
df = pd.merge(cctv, pop)
df.head()

,구별,CCTV댓수,최근증가율,인구수,외국인,고령자,외국인비율,고령자비율
0,종로구,1772,20.49,155106,9077,27605,5.85,17.80
1,용산구,2383,78.64,240665,14287,38884,5.94,16.16
2,성동구,3602,46.13,295767,6605,45591,2.23,15.41
3,광진구,2588,112.29,355306,12825,50815,3.61,14.30
4,동대문구,2497,34.85,352259,14219,61262,4.04,17.39


In [49]:
df.set_index('구별', inplace=True)
df.head()

,CCTV댓수,최근증가율,인구수,외국인,고령자,외국인비율,고령자비율
구별,,,,,,,
종로구,1772,20.49,155106,9077,27605,5.85,17.80
용산구,2383,78.64,240665,14287,38884,5.94,16.16
성동구,3602,46.13,295767,6605,45591,2.23,15.41
광진구,2588,112.29,355306,12825,50815,3.61,14.30
동대문구,2497,34.85,352259,14219,61262,4.04,17.39


In [51]:
df.to_csv('서울시_CCTV_인구_병합데이터.csv')